In [64]:
import sys
import os
import re
from query_rdfs import *
import itertools
import random

sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))
current_dir = os.path.dirname("__file__")
current_dir = current_dir if current_dir is not '' else '.'
# directory to scan for any txt files
data_dir_path = current_dir + '/rule'

known = {"name": "Che Viet Hai", "height": 170, "weight": 55, "gender": "nam", 
         "age": 17, "PhysicalActiveLevel": "Hoạt động vừa", "target": "giữ cân",
         "ListIllness": "Bệnh_béo_phì"}
data_rule = []
function = []

In [65]:
with open(data_dir_path + "/rule.txt", "r", encoding="utf8") as f:
    for line in f:
        data_rule.append(line)
with open(data_dir_path + "/function.txt", "r", encoding="utf8") as f:
    for line in f:
        function.append(line)

In [7]:
print(function)

['\ufeffbmr,nu=447.593+(9.247*weight)+(3.098*height)-(4.33*age)\n', 'bmr,nam=88.362+(13.397 *weight)+(4.799*height)-(5.677*age)\n', 'bmi=weight/(height*height)\n', 'calo=BMRvalue*PhysicalActiveValue\n', '1.2:Quá ít hoạt động\n', '1.375:Ít hoạt động\n', '1.55:Hoạt động vừa\n', '1.725:Hoạt động nhiều\n', '1.9:Hoạt động cường độ cao']


# BMI function

In [8]:
def BMIvalueCalc(height, weight):
    height/=100
    return round(eval(function[2].split("=")[1]),2)

def BMIrangeCalc(bmi_value):
    return BMI_range_calc(bmi_value)

def BMIlevelCalc(bmi_range):
    return has_value(bmi_range)["value"]

# BMR function

In [63]:
def BMRvalueOfMaleCalc(height, weight, age):
    result = function[1].split("=")[1]
    return round(eval(result), 2)

def BMRvalueOfFemaleCalc(height, weight, age):
    function[1] = function[1].split("=")[1]
    return round(eval(result), 2)

# print (BMIvalueCalc(height,weight)) 

# PA function

In [10]:
def PAvalueCalc(PhysicalActiveLevel):
    for count in range(4,9):
        result = function[count].split(":")
        if result[1].endswith("\n"):
            result[1] = result[1][:-1]
        if  result[1] == PhysicalActiveLevel:
            return float(result[0])

# CaloCalc function

In [11]:
def CaloCalc(BMRvalue, PhysicalActiveValue, target):
    result = round(eval(function[3].split("=")[1]),2)
    if target == "tăng cân":
        result +=500
    elif target == "giảm cân":
        result -=500
    return result

def CaloToKeepWeightCalc(calo_per_day):
    # breakfast: 26%, #Lunch: 40%, # Dinner: 34%
    return [round(calo_per_day*26/100, 2), 
            round(calo_per_day*40/100, 2), round(calo_per_day*34/100, 2)]    

# ListFood function

In [39]:
def FindAvoidFood(list_illness):
    return avoided_group_food(list_illness) # return array

def FindNeedFood(list_illness):
    return needed_group_food(list_illness)

def FindLimitFood(list_illness):
    return limit_food(list_illness)

def FilterFood(list_avoid_food, list_need_food, 
               list_limit_food):
    result = {}
    list_all_food = []
    tamp_list = [food.split("|") for food in all_food()]
    set_avoid_food = set(list_avoid_food)
    for food in tamp_list:
        if food[0] not in set_avoid_food:
            list_all_food.append(food)
    for food_1 in list_all_food:
        if food_1[0] in list_need_food:
            food_1.append(2)
            result[food_1[0]] = food_1[1:]
        elif food_1[0] in list_limit_food:
            food_1.append(-1)
            result[food_1[0]] = food_1[1:]
        else:
            food_1.append(1)
            result[food_1[0]] = food_1[1:]
    return result

# Extract_rule_file Function

In [13]:
def extract_rule_left(rule_left):
    rule_left = re.findall(r'\[(.*?)\]',rule_left)
    flag = True
    for content in rule_left:
        content = content.split(",")
        if content[0] == "1":
            if content[1] not in known:
                flag = False
                break
        elif content[0] == "2":
            try:
                if known[content[1]] != content[2]:
                    flag = False
                    break
            except:
                flag = False
                break
    return flag
                
def extract_rule_right(rule_right):
    rule_right = rule_right.split(",", 1)
    flag = True
    rule_right[0] = rule_right[0][1:]
    rule_right[1] = rule_right[1][:-2]
    if rule_right[0] in known:
        flag = False
    else:
        funct = rule_right[1].split(':')[1]
        known[rule_right[0]] = eval(funct)
    return flag
    
def extract_rule(rule):
    flag = True
    rule_left, rule_right = rule.split("|")
    flag = extract_rule_left(rule_left)
    if flag is False:
        return flag
    else: 
        flag = extract_rule_right(rule_right)
    return flag

In [59]:
count = 0
while count <= 3:
    if extract_rule(data_rule[count]) == True:
        print("True")
        count = 0
    else: count+=1        

IndexError: list index out of range

In [62]:
known = {'name': 'Che Viet Hai', 'height': 170, 'weight': 55, 'gender': 'nam', 'age': 17, 'PhysicalActiveLevel': 'Hoạt động vừa', 'target': 'giữ cân', 'ListIllness': 'Bệnh_béo_phì', 'BMIvalue': 19.03, 'BMIrange': '18.50-24.99', 'BMIlevel': 'Bình_thường'}
print(function[1])
print(extract_rule(data_rule[13]))

88.362+(13.397 *weight)+(4.799*height)-(5.677*age)

False


In [43]:
print(known)

{'name': 'Che Viet Hai', 'height': 170, 'weight': 55, 'gender': 'nam', 'age': 17, 'PhysicalActiveLevel': 'Hoạt động vừa', 'target': 'giữ cân', 'ListIllness': 'Bệnh_béo_phì', 'BMIvalue': 19.03, 'BMIrange': '18.50-24.99', 'BMIlevel': 'Bình_thường', 'BMRvalue': 1544.52, 'PhysicalActiveValue': 1.55, 'CaloPerDay': 2394.01, 'CaloForEachMeal': [622.44, 957.6, 813.96], 'ListAvoidFood': ['Peanut', 'Bánh_mì', 'Khoai_mì', 'Khoai_tây'], 'ListNeedFood': ['Bắp', 'Cà_rốt', 'Mực', 'Nho', 'Dưa_hấu', 'Táo'], 'ListLimitFood': [], 'ListFood': {'Almond': ['Cereal', '130', '', 1], 'Butter': ['Dairy_Products', '716', '', 1], 'Nut': ['Cereal', '134', '', 1], 'Sữa_bò': ['Dairy_Products', '70', "['Cam', 'Trứng_gà', 'Trứng_ngỗng', 'Trứng_vịt']", 1], 'Sữa_đặc': ['Dairy_Products', '320', '', 1], 'Walnut': ['Cereal', '137', '', 1], 'Atiso': ['Vegetables', '47', "['Thịt_chó']", 1], 'Bí_đỏ': ['Vegetables', '559', "['Cá_hồi', 'Lươn', 'Thịt_cừu', 'Tôm']", 1], 'Bơ': ['Fruits', '160', '', 1], 'Bưởi': ['Fruits', '42', "['

In [121]:
def find_type(list_food, types):
    result = {}
    for food in list_food:
        if list_food[food][0] == types:
            result[food] = list_food[food]
    return result

#rule_meal = [0.1, 1]
def caculate_gram(rule_meal, calor_food, calor_per_meal): #return 
    result = None
    
    calor_food_for_meal = rule_meal[0] * calor_per_meal
    portion = round(calor_food_for_meal / calor_food * 100, 1)
    
    result = [calor_food_for_meal, portion]
    return result

def food_not_use_together(list_food, meal):
    list_food_name = []
    list_food_not_use = set()
    for item in meal:
        for food_name in item:
            list_food_name.append(food_name)
            food_not_use = re.findall(r'\[(.*?)\]',list_food[food_name][2])
            if food_not_use != []:
                food_not_use = food_not_use[0].split(",")
                food_not_use = list(map(lambda x: x.replace("'", ""), food_not_use))
                for item in food_not_use:
                    list_food_not_use.add(item.strip())
    for food in list_food_name:
        if food in list_food_not_use:
            return False
    return True

def nCi(n_array, number):
    result = None

    result = list(itertools.combinations(n_array,number))
    return result

In [122]:
def FindListBreakFast(data):
    # Declear variable
    list_food = data["ListFood"]
    calor_each_meal = data["CaloForEachMeal"][0]
    print("BreakFast: ", calor_each_meal)
    result = []
    food_fix = {}
    food_next = []
    structure_breakfast = list_structure_breakfast(list_food)
    print(structure_breakfast)
    
    element = random.choice(structure_breakfast)
    print("element: ", element)
    units = {}
    for key in element:
        if key == "fix" :
            if len(element["fix"][0]) == 1:
                food_name = element[key][0][0]
                rule = element[key][1:]
                calor_food = float(list_food[food_name][1])
                food_fix[food_name] = caculate_gram(rule, calor_food, calor_each_meal)
        else:
            types = key
            rule = element[key]
#             print(types)
            list_food_types = find_type(list_food, types)
            sorted_by_value = dict(sorted(list_food_types.items(), 
                                          key=lambda kv: kv[1][3], reverse=True))
            best_food = []
            for count, units in enumerate(sorted_by_value):
                if count > 2: break 
                else: # pick three foods have best score
                    best_food.append(units)
            combination = nCi(best_food, rule[1])
#             print(combination)
            for index, element_combination in enumerate (combination):
                tamp = {}
                for food_name in element_combination:
                    calor_food = float(list_food[food_name][1])
                    tamp[food_name] = caculate_gram(rule, calor_food, calor_each_meal)
                    combination[index] = tamp
            food_next.append(combination)
    
    # create result
    food_next.append([food_fix])
#     print('food_next: ', food_next)
    tamp = list(itertools.product(*food_next))
    for item in tamp:
        if food_not_use_together(list_food, item):
            result.append(item)
    return result

def FindlistDinner(data):
    structure_dinner = list_structure_dinner(list_food["only_food_name"])
    print(structure_dinner)
    
def FindlistLunch(data):
    # Declear variable
    list_food = data["ListFood"]
    calor_each_meal = data["CaloForEachMeal"][1]
    print("lunch: ", calor_each_meal)
    result = []
    food_fix = {}
    food_next = []
    structure_lunch = list_structure_lunch(list_food)
    
    element = random.choice(structure_lunch)
    print("element: ", element)
    units = {}
    for key in element:
        if key == "fix" :
            if len(element["fix"][0]) == 1:
                food_name = element[key][0][0]
                rule = element[key][1:]
                calor_food = float(list_food[food_name][1])
                food_fix[food_name] = caculate_gram(rule, calor_food, calor_each_meal)
        else:
            types = key
            rule = element[key]
#             print(types)
            list_food_types = find_type(list_food, types)
            sorted_by_value = dict(sorted(list_food_types.items(), 
                                          key=lambda kv: kv[1][3], reverse=True))
            best_food = []
            for count, units in enumerate(sorted_by_value):
                if count > 2: break 
                else: # pick three foods have best score
                    best_food.append(units)
            combination = nCi(best_food, rule[1])
#             print(combination)
            for index, element_combination in enumerate (combination):
                tamp = {}
                for food_name in element_combination:
                    calor_food = float(list_food[food_name][1])
                    tamp[food_name] = caculate_gram(rule, calor_food, calor_each_meal)
                    combination[index] = tamp
            food_next.append(combination)
    
    # create result
    food_next.append([food_fix])
#     print('food_next: ', food_next)
    tamp = list(itertools.product(*food_next))
    for item in tamp:
        if food_not_use_together(list_food, item):
            result.append(item)
    return result

In [123]:
print(len(FindListBreakFast(known)[1]))

BreakFast:  622.44
[]


IndexError: Cannot choose from an empty sequence

In [22]:
print ([key for key in known])

['name', 'height', 'weight', 'gender', 'age', 'PhysicalActiveLevel', 'target', 'ListIllness', 'BMIvalue', 'BMIrange', 'BMIlevel', 'BMRvalue', 'PhysicalActiveValue', 'CaloPerDay', 'CaloForEachMeal', 'ListAvoidFood', 'ListNeedFood', 'ListLimitFood', 'ListFood']


In [14]:
tamp = list(itertools.combinations(['ab','bc','dc'],1))

print(tamp)
print(list(itertools.combinations(['a','b','c'],1)))
[''.join(x) for x in itertools.combinations(['a','b','c'],2)]

[('ab',), ('bc',), ('dc',)]
[('a',), ('b',), ('c',)]


['ab', 'ac', 'bc']

In [17]:
def nCi(n_array, number):
    result = None
    
    result = list(itertools.combinations(n_array,number))
    return result

a = nCi([['a','b','c'], ['d','e'],['f', 'g']], 3)
print (a)

[(['a', 'b', 'c'], ['d', 'e'], ['f', 'g'])]


In [87]:
def food_not_use_together(list_food, meal):
    list_food_name = []
    list_food_not_use = set()
    for item in meal:
        for food_name in item:
            list_food_name.append(food_name)
            food_not_use = re.findall(r'\[(.*?)\]',list_food[food_name][2])
            food_not_use = food_not_use[0].split(",")
            food_not_use = list(map(lambda x: x.replace("'", ""), food_not_use))
            for item in food_not_use:
                list_food_not_use.add(item.strip())
    for food in list_food_name:
        if food in list_food_not_use:
            return False
    return True
print(food_not_use_together(known['ListFood'], [{'Tôm': 2}, {'Bí_đỏ': 3}]))

False


In [67]:
food_next = [[[{'Mực': [143.64, 181.8]}, {'Cua': [143.64, 169.0]}, {'Cá_chép': [143.64, 113.1]}], [{'Cà_rốt': [95.76, 233.6]}, {'Atiso': [95.76, 203.7]}, {'Bí_đỏ': [95.76, 17.1]}]]
for item in food_next:
             
    if item

SyntaxError: invalid syntax (<ipython-input-67-bc1aff6265cb>, line 2)

In [78]:
#rule_meal = [0.1, 1]


def caculate_gram(rule_meal, calor_food, calor_per_meal): #return 
    result = None
    
    calor_food_for_meal = rule_meal[0] * calor_per_meal
    portion = round(calor_food_for_meal / calor_food * 100, 1)
    
    result = [calor_food_for_meal, portion]
    return result

print(caculate_gram([0.1, 1], 60, 120))

[12.0, 20.0]


In [29]:
a = [[{'Cơm': [3,2]}], [{'Thịt_bò': [95.76, 37.7]}, {'Thịt_cừu': [95.76, 34.0]}, {'Thịt_dê': [95.76, 78.5]}], [{'Mực': [95.76, 121.2]}, {'Cua': [95.76, 112.7]}, {'Cá_chép': [95.76, 75.4]}], [{'Cà_rốt': [95.76, 233.6]}, {'Atiso': [95.76, 203.7]}, {'Bí_đỏ': [95.76, 17.1]}], [{'Bắp': [191.52, 222.7]}, {'Dưa_hấu': [191.52, 638.4]}, {'Nho': [191.52, 64.1]}]]
result = list(itertools.product(*a))
print(len(result))

81


In [45]:
def caculate_menu_score(menu, list_food):
    result = None
    score_real = 0
    score_abstract = 0
    confident = None
    ideal_score = 2
    for meal in menu:
        for item in meal:
            for food in item:
                score_real += list_food[food][3] # food_score
                score_abstract += ideal_score
     
    print(score_abstract)
    print(score_real)
    confident = round(score_real / score_abstract * 100, 2)
    print(confident)
    menu = (menu, confident)
    return menu

In [4]:
menu = ([['Cơm', 226.16, 110.9], ['Trứng_gà', 84.81, 22], ['Thịt_dê', 84.81, 24]], [['Cơm', 226.16, 110.9], ['Trứng_gà', 84.81, 22], ['Thịt_dê', 84.81, 24], ['Thịt_bò', 33, 55]], [['Dưa_hấu', 226.16, 110.9], ['Đậu_đỏ', 84.81, 22], ['Hẹ', 84.81, 24], ['Kiwi', 33, 55]])

In [9]:
def caculate_calo(menu):
    result = 0
    for meal in menu:
        for food in meal:
            result += food[1]

    return result

def portion_meal(menu, calo_menu):
    result = []
    for meal in menu:
        meal_calo = 0
        for food in meal:
            meal_calo += food[1]
        tamp = round(meal_calo / calo_menu * 100, 2)
        result.append(tamp)

    return result
print(portion_meal(menu, caculate_calo(menu)))

[31.58, 34.21, 34.21]


In [57]:
f.write("Những chất dinh dưỡng: ", str(", ".join(result['Has_Nutrient'])))

None
[(3, 2, 3), (3, 3, 4)]


In [10]:
one = 22
if one in range(22, 30+1):
    print("s")

s
